In [9]:
import pandas as pd

In [3]:
import sqlite3

conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()


In [5]:
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

In [17]:
providers['City'] = providers['City'].fillna('Unknown')
receivers['City'] = receivers['City'].fillna('Unknown')
food_listings['Location'] = food_listings['Location'].fillna('Unknown')

In [7]:
print(providers.isnull().sum())

Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64


In [9]:
food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'])

In [19]:
providers['Provider_ID'] = providers['Provider_ID'].astype(int)
receivers['Receiver_ID'] = receivers['Receiver_ID'].astype(int)
food_listings['Food_ID'] = food_listings['Food_ID'].astype(int)
claims['Claim_ID'] = claims['Claim_ID'].astype(int)

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Food_Listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date DATE,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES Providers (Provider_ID)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings (Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers (Receiver_ID)
)
""")


In [11]:

providers = pd.read_csv("providers_data.csv")
providers.to_sql("Providers", conn, if_exists="append", index=False)

receivers = pd.read_csv("receivers_data.csv")
receivers.to_sql("Receivers", conn, if_exists="append", index=False)

food_listings = pd.read_csv("food_listings_data.csv")
food_listings.to_sql("Food_Listings", conn, if_exists="append", index=False)

claims = pd.read_csv("claims_data.csv")
claims.to_sql("Claims", conn, if_exists="append", index=False)


1000

In [21]:
conn.commit()

In [23]:
# Step 3 ke liye same database connection use karenge
import pandas as pd
import sqlite3

conn = sqlite3.connect("food_wastage.db")

# 1. Kitne providers aur receivers har city me hain
query1 = """
SELECT City, COUNT(*) AS Total_Providers
FROM Providers
GROUP BY City
"""
print("Providers per City:")
display(pd.read_sql(query1, conn))

query2 = """
SELECT City, COUNT(*) AS Total_Receivers
FROM Receivers
GROUP BY City
"""
print("Receivers per City:")
display(pd.read_sql(query2, conn))

# 2. Kaunsa provider type sabse zyada food contribute karta hai
query3 = """
SELECT Type, COUNT(*) AS Contribution_Count
FROM Providers
GROUP BY Type
ORDER BY Contribution_Count DESC
"""
display(pd.read_sql(query3, conn))

# 3. Kisi specific city ke providers ka contact info
city_name = "Delhi"  # Example
query4 = f"""
SELECT Name, Contact
FROM Providers
WHERE City = '{city_name}'
"""
display(pd.read_sql(query4, conn))

# 4. Kaunse receivers ne sabse zyada food claim kiya
query5 = """
SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
FROM Receivers r
JOIN Claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.Name
ORDER BY Total_Claims DESC
"""
display(pd.read_sql(query5, conn))

# 5. Total quantity of food available
query6 = "SELECT SUM(Quantity) AS Total_Food_Quantity FROM Food_Listings"
display(pd.read_sql(query6, conn))

# 6. Kis city me sabse zyada food listings hain
query7 = """
SELECT Location, COUNT(*) AS Total_Listings
FROM Food_Listings
GROUP BY Location
ORDER BY Total_Listings DESC
"""
display(pd.read_sql(query7, conn))

# 7. Most common food types
query8 = """
SELECT Food_Type, COUNT(*) AS Count
FROM Food_Listings
GROUP BY Food_Type
ORDER BY Count DESC
"""
display(pd.read_sql(query8, conn))

# 8. Kitne food claims har item ke liye hue
query9 = """
SELECT f.Food_Name, COUNT(c.Claim_ID) AS Total_Claims
FROM Food_Listings f
LEFT JOIN Claims c ON f.Food_ID = c.Food_ID
GROUP BY f.Food_Name
ORDER BY Total_Claims DESC
"""
display(pd.read_sql(query9, conn))

# 9. Kis provider ke paas sabse zyada successful claims hain
query10 = """
SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
FROM Providers p
JOIN Food_Listings f ON p.Provider_ID = f.Provider_ID
JOIN Claims c ON f.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
"""
display(pd.read_sql(query10, conn))

# 10. Claims ka percentage (Completed, Pending, Cancelled)
query11 = """
SELECT Status, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Claims) AS Percentage
FROM Claims
GROUP BY Status
"""
display(pd.read_sql(query11, conn))

# 11. Average quantity of food claimed per receiver
query12 = """
SELECT r.Name, AVG(f.Quantity) AS Avg_Quantity_Claimed
FROM Receivers r
JOIN Claims c ON r.Receiver_ID = c.Receiver_ID
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
GROUP BY r.Name
"""
display(pd.read_sql(query12, conn))

# 12. Most claimed meal type
query13 = """
SELECT Meal_Type, COUNT(*) AS Total_Claims
FROM Food_Listings f
JOIN Claims c ON f.Food_ID = c.Food_ID
GROUP BY Meal_Type
ORDER BY Total_Claims DESC
"""
display(pd.read_sql(query13, conn))

# 13. Total quantity of food donated by each provider
query14 = """
SELECT p.Name, SUM(f.Quantity) AS Total_Donated
FROM Providers p
JOIN Food_Listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC
"""
display(pd.read_sql(query14, conn))


Providers per City:


,City,Total_Providers
0,Adambury,1
1,Adamsview,1
2,Adamsville,1
3,Aguirreville,1
4,Alexanderchester,1
...,...,...
958,Wrightville,1
959,Yatesside,1
960,Youngchester,1
961,Zimmermanton,1


Receivers per City:


,City,Total_Receivers
0,Aaronshire,1
1,Adamland,1
2,Aguilarbury,1
3,Aguilarstad,1
4,Alexanderbury,1
...,...,...
961,Woodardview,1
962,Woodsfurt,1
963,Wrightland,1
964,Wyattton,1


,Type,Contribution_Count
0,Supermarket,262
1,Grocery Store,256
2,Restaurant,246
3,Catering Service,236


...

,Name,Successful_Claims
0,Barry Group,5
1,Miller Inc,4
2,"Harper, Blake and Alexander",4
3,Butler-Richardson,4
4,"Barnes, Castro and Curtis",4
...,...,...
243,"Anderson, Nicholson and Bruce",1
244,Allen-Mccullough,1
245,"Allen, Zimmerman and Harrington",1
246,Alexander PLC,1


,Status,Percentage
0,Cancelled,33.6
1,Completed,33.9
2,Pending,32.5


,Name,Avg_Quantity_Claimed
0,Aaron Keller,39.000000
1,Aaron Rios,21.000000
2,Aaron Scott,45.000000
3,Abigail Crawford,25.666667
4,Adam Browning,5.000000
...,...,...
615,William Barnes,47.000000
616,William Fox,12.000000
617,William Frederick,21.400000
618,Yvette Huffman,45.000000


,Meal_Type,Total_Claims
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


,Name,Total_Donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
...,...,...
623,Reyes and Sons,1
624,Norris-Townsend,1
625,Mcgee PLC,1
626,"Martinez, Armstrong and Carroll",1
